In [4]:
import tensorflow
import gym
import random
import atari_py
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Convolution2D
from tensorflow.keras.optimizers import Adam
from rl.agents import DQNAgent
from rl.memory import SequentialMemory
from rl.policy import BoltzmannQPolicy, EpsGreedyQPolicy



from gym import Env
from gym.spaces import Discrete, Box
import random

import pandas as pd
import tensorflow as tf
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder


In [60]:
all_df = pd.read_csv("train.csv")
all_df = all_df.dropna()





In [6]:
def room_to_numeric(cabin):
    sum = 0
    for i in range (0, len(cabin)  ):
        sum += (ord(cabin[i]))
    return sum

175

In [61]:
for j in range (0,len(all_df)):
    all_df.iloc[j,10] = room_to_numeric(all_df.iloc[j,10] )
    if(all_df.iloc[j,4] == "female"):
        all_df.iloc[j,4] = 1
    else:
        all_df.iloc[j,4] = 0


all_df

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",1,38.0,1,0,PC 17599,71.2833,176,C
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",1,35.0,1,0,113803,53.1000,217,S
6,7,0,1,"McCarthy, Mr. Timothy J",0,54.0,0,0,17463,51.8625,175,S
10,11,1,3,"Sandstrom, Miss. Marguerite Rut",1,4.0,1,1,PP 9549,16.7000,125,S
11,12,1,1,"Bonnell, Miss. Elizabeth",1,58.0,0,0,113783,26.5500,215,S
...,...,...,...,...,...,...,...,...,...,...,...,...
871,872,1,1,"Beckwith, Mrs. Richard Leonard (Sallie Monypeny)",1,47.0,1,1,11751,52.5542,172,S
872,873,0,1,"Carlsson, Mr. Frans Olof",0,33.0,0,0,695,5.0000,574,S
879,880,1,1,"Potter, Mrs. Thomas Jr (Lily Alexenia Wilson)",1,56.0,0,1,11767,83.1583,168,C
887,888,1,1,"Graham, Miss. Margaret Edith",1,19.0,0,0,112053,30.0000,168,S


In [65]:
all_df = all_df.drop(all_df.columns[[0,3,8,11]], axis=1)

In [78]:
all_df.iloc[1].tolist()[1:8]

[1, 1, 35.0, 1, 0, 53.1, 217]

In [86]:
class ShowerEnv(Env):
    def __init__(self):
        self.action_space = Discrete(2)
        self.duration = 0
        self.state = all_df.iloc[self.duration].tolist()[1:8]
        
        

    def step(self, action):
        self.duration +=1

        self.state = all_df.iloc[self.duration].tolist()[1:8]
        


        if all_df.iloc[self.duration].tolist()[0] == self.action:
            reward =1
        else:
            reward = -1

        if self.duration >= 10000:
            done = True
        else:
            done = False

        info = {}

        return self.state, reward, done, info

    def render(self):
        pass

    def reset(self):
        self.duration =0
        self.state = all_df.iloc[self.duration].tolist()[1:8]
        
        return self.state

    

In [87]:
env = ShowerEnv()

In [89]:
env.action_space.sample()

1

In [108]:
episodes = 10
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0
    
    while not done:
        env.render()
        action = env.action_space.sample()
        n_state, reward, done, info = env.step(action)
        score += reward
    print('Episode:{} score:{}'.format(episode, score))
env.close()

AttributeError: 'ShowerEnv' object has no attribute 'action'

In [91]:
states = env.observation_space.shape
print(states)
actions = env.action_space.n

AttributeError: 'NoneType' object has no attribute 'shape'

In [104]:
def build_model():
    model = Sequential()
    
    model.add(Dense(48, activation='relu', input_shape = [8,1]))
    
    model.add(Dense(21, activation = 'relu'))
    model.add(Dense(2, activation = 'linear'))
    return model


In [105]:
model = build_model()



In [106]:
def build_agent(model, actions):
    policy = BoltzmannQPolicy()
    
    memory = SequentialMemory(limit =100, window_length =1)
    
    dqn = DQNAgent(model = model,
                   memory=memory,
                   policy=policy, 
                   
                   nb_actions = actions,
                   nb_steps_warmup =100,
                   target_model_update=1e-2
                  )
    return dqn

    
    

In [107]:
dqn = build_agent(model, 2)
dqn.compile(Adam(lr=1e-2), metrics=['mae'])
dqn.fit(env, nb_steps = 100000, visualize=False, verbose =1)

ValueError: Model output "Tensor("dense_7/BiasAdd:0", shape=(None, 8, 2), dtype=float32)" has invalid shape. DQN expects a model that has one dimension for each action, in this case 2.

In [18]:
scores = dqn.test(env, nb_episodes=100, visualize=False)
print(np.mean(scores.history['episode_reward']))

Testing for 100 episodes ...
Episode 1: reward: 30.000, steps: 60
Episode 2: reward: 52.000, steps: 60
Episode 3: reward: 56.000, steps: 60
Episode 4: reward: 46.000, steps: 60
Episode 5: reward: 20.000, steps: 60
Episode 6: reward: 2.000, steps: 60
Episode 7: reward: 34.000, steps: 60
Episode 8: reward: 56.000, steps: 60
Episode 9: reward: 60.000, steps: 60
Episode 10: reward: -14.000, steps: 60
Episode 11: reward: 58.000, steps: 60
Episode 12: reward: 36.000, steps: 60
Episode 13: reward: 6.000, steps: 60
Episode 14: reward: 18.000, steps: 60
Episode 15: reward: -6.000, steps: 60
Episode 16: reward: 38.000, steps: 60
Episode 17: reward: 30.000, steps: 60
Episode 18: reward: -50.000, steps: 60
Episode 19: reward: 16.000, steps: 60
Episode 20: reward: 26.000, steps: 60
Episode 21: reward: -10.000, steps: 60
Episode 22: reward: -16.000, steps: 60
Episode 23: reward: 2.000, steps: 60
Episode 24: reward: 26.000, steps: 60
Episode 25: reward: 60.000, steps: 60
Episode 26: reward: -52.000, 